<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/AlgoTradingTest01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sourcing Alpaca Credentials

In [1]:
credentials = open("/content/drive/MyDrive/Colab Notebooks/alpaca.txt","r")
lines = credentials.readlines()
consumer_key = lines[0].rstrip().replace("alpaca_key=","")
consumer_secret = lines[1].rstrip().replace("alpaca_secret=","")
#print(consumer_secret)


In [2]:
!pip install alpaca-trade-api requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.9 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.5
    Uninstalling msgpack-1.0.5:
      Successfully uninstalled msgpack-1.0.5
  Attempting uninstall: PyYAML
   

In [32]:
import pandas as pd
from alpaca_trade_api import REST, Stream, TimeFrame,TimeFrameUnit
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta

API_KEY = consumer_key
API_SECRET = consumer_secret

def fetch_news_summary(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        summary = soup.find('p').text
        return summary.strip()
    except Exception as e:
        print(f"Error fetching news summary: {e}")
        return None

def get_news_data(ALPACA_API_KEY,ALPACA_SECRET_KEY,ticker):
    # Get News for last 24 hrs
    today = datetime.today()- timedelta(1)
    yesterday = today - timedelta(2)
    today = today.strftime('%Y-%m-%d')
    yesterday = yesterday.strftime('%Y-%m-%d')

    # Initialize Alpaca API
    # Get the news data
    rest_client = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    news_list = rest_client.get_news(ticker,yesterday,today)
    bars = rest_client.get_bars(ticker, TimeFrame(59, TimeFrameUnit.Minute), yesterday,today, adjustment='raw').df

    # Convert the news data into a pandas DataFrame
    data = {
        'Title': [],
        'Published': [],
        'Source': [],
        'Url': [],
        'Summary': []
    }

    for news in news_list:
        data['Title'].append(news.headline)
        data['Published'].append(news.created_at.strftime('%Y-%m-%d %H:%M'))
        data['Source'].append(news.source)
        data['Url'].append(news.url)
        data['Summary'].append(fetch_news_summary(news.url))

    df = pd.DataFrame(data)
    return df,bars

if __name__ == "__main__":
    news_df,bars_data = get_news_data(API_KEY,API_SECRET,"AAPL")

In [33]:
bars_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2023-07-31 07:08:00+00:00,195.8900,195.9000,195.3900,195.5900,8324,290,195.680316
2023-07-31 08:07:00+00:00,195.6900,195.8100,195.5900,195.7800,13394,333,195.715902
2023-07-31 09:06:00+00:00,195.8000,195.9500,195.5700,195.7000,13007,468,195.784594
2023-07-31 10:05:00+00:00,195.8300,195.9800,195.7400,195.9800,28030,719,195.885564
2023-07-31 11:04:00+00:00,195.9800,196.5400,195.7885,195.9500,166509,4280,196.122664
2023-07-31 12:03:00+00:00,196.3900,196.7100,195.5100,196.5400,204730,4278,196.460514
2023-07-31 13:02:00+00:00,196.5500,196.6000,195.3900,195.4200,5594175,106382,195.933186
2023-07-31 14:01:00+00:00,195.4200,196.4350,195.2600,196.2600,5450785,180651,195.970122
2023-07-31 15:00:00+00:00,196.2700,196.4900,195.9500,196.1300,3573413,46216,196.231030
